In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnull, isnan 
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("ChurnPredictionLargeMemory") \
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()
    
print("SparkSession berhasil dibuat dengan konfigurasi baru!")

SparkSession berhasil dibuat dengan konfigurasi baru!


In [ ]:
try:
    path = 'dataset/'
    
    print("Memuat dataset sebagai DataFrame Spark...")
    
    # Memuat semua file
    train_df = spark.read.csv(path + 'train.csv', header=True, inferSchema=True)
    train2_df = spark.read.csv(path + 'train_v2.csv', header=True, inferSchema=True)
    members_df = spark.read.csv(path + 'members_v3.csv', header=True, inferSchema=True)
    transactions_df = spark.read.csv(path + 'transactions.csv', header=True, inferSchema=True)
    transactions2_df = spark.read.csv(path + 'transactions_v2.csv', header=True, inferSchema=True)
    user_logs_df = spark.read.csv(path + 'user_logs.csv', header=True, inferSchema=True)
    user_logs2_df = spark.read.csv(path + 'user_logs_v2.csv', header=True, inferSchema=True)
    
    print("Semua dataset berhasil dimuat.")
    
    # Tampilkan jumlah baris untuk validasi
    print(f"\nJumlah baris:")
    print(f"train: {train_df.count():,}")
    print(f"train_v2: {train2_df.count():,}")
    print(f"members_v3: {members_df.count():,}")
    print(f"transactions: {transactions_df.count():,}")
    print(f"transactions_v2: {transactions2_df.count():,}")
    # .count() pada user_logs 
    print(f"user_logs: {user_logs_df.count():,}")
    print(f"user_logs_v2: {user_logs2_df.count():,}")


except Exception as e:
    print(f"ERROR: {e}")

Memuat dataset sebagai DataFrame Spark...
Semua dataset berhasil dimuat.

Jumlah baris:
train: 992,931
train_v2: 970,960
members_v3: 6,769,473
transactions: 21,547,746
transactions_v2: 1,431,009
user_logs: 392,106,543
user_logs_v2: 18,396,362


In [ ]:
from pyspark.sql.functions import col

path = 'dataset/train'
    

train_df = spark.read.csv(path + 'train.csv', header=True, inferSchema=True)
train2_df = spark.read.csv(path + 'train_v2.csv', header=True, inferSchema=True)
print("\nMenggabungkan dataset 'train' dan 'train_v2'...")

# Langkah 1: Filter train2_df untuk mendapatkan msno yang belum ada di train_df
# Kita menggunakan 'left_anti' join untuk ini.
# Ini seperti mengatakan: "Berikan aku semua baris dari train2_df yang msno-nya TIDAK ADA di train_df"
train2_unique_df = train2_df.join(train_df, on="msno", how="left_anti")

# Langkah 2: Gabungkan train_df asli dengan baris unik dari train2_df
# Gunakan unionByName untuk keamanan, memastikan kolom digabungkan berdasarkan nama, bukan posisi.
train_combined_df = train_df.unionByName(train2_unique_df)

# Verifikasi hasil penggabungan
print("\nProses penggabungan selesai.")
print(f"Jumlah baris train_df asli: {train_df.count():,}")
print(f"Jumlah baris train2_df yang unik (baru): {train2_unique_df.count():,}")
print(f"Jumlah baris total setelah digabung: {train_combined_df.count():,}")

# buat cek jumlahnya cocok: train_df.count() + train2_unique_df.count()

print("\nContoh hasil penggabungan:")
train_combined_df.show(5)

output_path_single_file = 'dataset/train_combined_single.csv'

print(f"\nMenyimpan DataFrame sebagai SATU file CSV ke: {output_path_single_file}")

train_combined_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path_single_file)

print("Penyimpanan sebagai file tunggal selesai.")

Memuat dataset sebagai DataFrame Spark...

Menggabungkan dataset 'train' dan 'train_v2'...

Proses penggabungan selesai.
Jumlah baris train_df asli: 992,931
Jumlah baris train2_df yang unik (baru): 89,259
Jumlah baris total setelah digabung: 1,082,190

Contoh hasil penggabungan:
+--------------------+--------+
|                msno|is_churn|
+--------------------+--------+
|waLDQMmcOu2jLDaV1...|       1|
|QA7uiXy8vIbUSPOkC...|       1|
|fGwBva6hikQmTJzrb...|       1|
|mT5V8rEpa+8wuqi6x...|       1|
|XaPhtGLk/5UvvOYHc...|       1|
+--------------------+--------+
only showing top 5 rows

Menyimpan DataFrame sebagai SATU file CSV ke: dataset/train_combined_single.csv
Penyimpanan sebagai file tunggal selesai.


In [3]:
path = 'dataset/transactions/'
transactions_df = spark.read.csv(path + 'transactions.csv', header=True, inferSchema=True)
transactions2_df = spark.read.csv(path + 'transactions_v2.csv', header=True, inferSchema=True)

print("\nMenggabungkan dataset 'transactions' dan 'transactions_v2'...")

# Gabungkan kedua DataFrame transaksi menjadi satu
transactions_full_df = transactions_df.unionByName(transactions2_df)

# Tentukan kolom-kolom yang menjadi kunci unik sebuah transaksi
key_columns = ['msno', 'transaction_date', 'membership_expire_date', 'actual_amount_paid']

# Buang baris yang duplikat berdasarkan 'key_columns'
transactions_combined_df = transactions_full_df.distinct()

# Verifikasi
print("\nProses penggabungan transaksi selesai.")
print(f"Jumlah baris transactions_df: {transactions_df.count():,}")
print(f"Jumlah baris transactions2_df: {transactions2_df.count():,}")
print(f"Jumlah baris total sebelum deduplikasi: {transactions_full_df.count():,}")
print(f"Jumlah baris total setelah deduplikasi: {transactions_combined_df.count():,}")

print("\nContoh hasil penggabungan transaksi:")
transactions_combined_df.show(5)

output_path_transactions = 'dataset/transactions_combined_single.csv'

print(f"\nMenyimpan DataFrame transaksi gabungan sebagai SATU file CSV ke: {output_path_transactions}")

transactions_combined_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path_transactions)

print("Penyimpanan data transaksi sebagai file tunggal selesai.")


Menggabungkan dataset 'transactions' dan 'transactions_v2'...

Proses penggabungan transaksi selesai.
Jumlah baris transactions_df: 21,547,746
Jumlah baris transactions2_df: 1,431,009
Jumlah baris total sebelum deduplikasi: 22,978,755
Jumlah baris total setelah deduplikasi: 22,975,416

Contoh hasil penggabungan transaksi:
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|NgBigmhVpkPNDi5Zm...|               41|               30|            149|               149|            1|        20151208|              20160108|        0|
|+kBlp0KK3Ys0KLq3L...|               35|   

In [ ]:
path = 'dataset/user_logs/'
user_logs_df = spark.read.csv(path + 'user_logs.csv', header=True, inferSchema=True)
user_logs2_df = spark.read.csv(path + 'user_logs_v2.csv', header=True, inferSchema=True)

user_logs_combined_df = user_logs_df.unionByName(user_logs2_df)
user_logs_combined_df_unique = user_logs_combined_df.distinct()

print("\nProses penggabungan user_logs selesai.")
print(f"Jumlah baris user_logs_df: {user_logs_df.count():,}")
print(f"Jumlah baris user_logs2_df: {user_logs2_df.count():,}")
print(f"Jumlah baris user_logs_df yang digabung: {user_logs_combined_df.count():,}")
print(f"Jumlah baris user_logs2_df yang unik (baru): {user_logs_combined_df_unique.count():,}")

print("\nContoh hasil penggabungan user_logs:")
user_logs_combined_df.show(5)

output_path_user_logs = 'dataset/user_logs_combined_single.csv'

print(f"\nMenyimpan DataFrame user_logs gabungan sebagai SATU file CSV ke: {output_path_user_logs}")

user_logs_combined_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv(output_path_user_logs)

print("Penyimpanan data user_logs sebagai file tunggal selesai.")


Proses penggabungan user_logs selesai.
Jumlah baris user_logs_df: 392,106,543
Jumlah baris user_logs2_df: 18,396,362
Jumlah baris user_logs_df yang digabung: 410,502,905
Jumlah baris user_logs2_df yang unik (baru): 410,502,905

Contoh hasil penggabungan user_logs:
+--------------------+--------+------+------+------+-------+-------+-------+----------+
|                msno|    date|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+--------------------+--------+------+------+------+-------+-------+-------+----------+
|rxIP2f2aN0rYNp+to...|20150513|     0|     0|     0|      0|      1|      1|   280.335|
|rxIP2f2aN0rYNp+to...|20150709|     9|     1|     0|      0|      7|     11|  1658.948|
|yxiEWwE9VR5utpUec...|20150105|     3|     3|     0|      0|     68|     36| 17364.956|
|yxiEWwE9VR5utpUec...|20150306|     1|     0|     1|      1|     97|     27| 24667.317|
|yxiEWwE9VR5utpUec...|20150501|     3|     0|     0|      0|     38|     38|  9649.029|
+--------------------+--------